In [1]:
import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 

from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack


2.0.1+cu117


/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 

from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack


2.0.1+cu117


/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_training='tokemonster'#'bert' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 10_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
if model_training=='bert':
    VOCAB_SIZE = 32_768  # from Cramming
    DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
    MODEL_MAX_SEQ_LEN = 128  # from Cramming
else:
    VOCAB_SIZE = 16_000  # tokenmonster
    # VOCAB_SIZE = 1_024  # tokenmonster
    DEVICE_BATCH_SIZE = 200 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
    MODEL_MAX_SEQ_LEN = 84  # token_monster

    # DEVICE_BATCH_SIZE=12
    # MODEL_MAX_SEQ_LEN = 1000


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from process_tokenizer import get_tokenizer
tokenizer,tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,model_training,MODEL_MAX_SEQ_LEN+2)


batch_size:  2000


Resolving data files: 100%|██████████| 53/53 [00:00<00:00, 123.82it/s]


loaded dataset!!


Feeding samples to tokenizer: 100%|██████████| 10000/10000 [00:00<00:00, 19739.04it/s]





Tokenizer trained in 900 milliseconds.
[MASK] 4


In [3]:
# Creating a RetNet model
import torch
from torchscale.architecture.config import RetNetConfig
from torchscale.architecture.retnet import RetNetDecoder
from torchscale.component.embedding import PositionalEmbedding, TextEmbedding


decoder_embed_dim=int(768*0.25)
decoder_ffn_embed_dim=int(1536*0.25)
recurrent_chunk_size=int(512*0.25)
encoder_embed_tokens=TextEmbedding(VOCAB_SIZE+8, decoder_embed_dim)
config = RetNetConfig(vocab_size=VOCAB_SIZE+8,
                    max_target_positions=MODEL_MAX_SEQ_LEN,
                    decoder_layers=12,
                    decoder_retention_heads=3,
                    decoder_embed_dim=decoder_embed_dim,
                    decoder_ffn_embed_dim=decoder_ffn_embed_dim,
                    recurrent_chunk_size=recurrent_chunk_size
                    )
generator = RetNetDecoder(config,encoder_embed_tokens)
# to_logits = torch.nn.Linear(decoder_embed_dim, VOCAB_SIZE+8)
# generator.to_logits = to_logits

#print number of parameters
total_params = sum(p.numel() for p in generator.parameters())
print(f'{total_params:,} total parameters.')

5,746,560 total parameters.


In [4]:
# prev_output_tokens = torch.zeros((2, MODEL_MAX_SEQ_LEN), dtype=torch.long)
# o=generator(
#     prev_output_tokens=prev_output_tokens,
#     token_embeddings=None,
#     features_only=True,
# )
# # logits=generator.to_logits(o[0])
# o[0].shape

In [5]:
decoder_embed_dim=int(768*0.75)
decoder_ffn_embed_dim=int(1536*0.75)
recurrent_chunk_size=int(512*0.75)

encoder_embed_tokens=TextEmbedding(VOCAB_SIZE+8, decoder_embed_dim)



config = RetNetConfig(vocab_size=VOCAB_SIZE+8,
                    max_target_positions=MODEL_MAX_SEQ_LEN,
                    decoder_layers=12,
                    decoder_retention_heads=3,
                    decoder_embed_dim=decoder_embed_dim,
                    decoder_ffn_embed_dim=decoder_ffn_embed_dim,
                    recurrent_chunk_size=recurrent_chunk_size                  
                    )
discriminator = RetNetDecoder(config,encoder_embed_tokens)
to_binary = torch.nn.Linear(decoder_embed_dim,1)
discriminator.to_binary = to_binary

#print number of parameters
total_params = sum(p.numel() for p in discriminator.parameters())
print(f'{total_params:,} total parameters.')

49,090,753 total parameters.


In [6]:
from base_electra_model import *
class Electra(nn.Module):
    def __init__(
        self,
        generator,
        discriminator,
        *,
        num_tokens = None,
        discr_dim = -1,
        discr_layer = -1,
        mask_prob = 0.15,
        replace_prob = 0.85,
        random_token_prob = 0.,
        mask_token_id = 2,
        pad_token_id = 0,
        mask_ignore_token_ids = [],
        disc_weight = 50.,
        gen_weight = 1.,
        temperature = 1.):
        super().__init__()

        self.generator = generator
        self.discriminator = discriminator

        # if discr_dim > 0:
        #     self.discriminator = nn.Sequential(
        #         HiddenLayerExtractor(discriminator, layer = discr_layer),
        #         nn.Linear(discr_dim, 1)
        #     )

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens
        self.random_token_prob = random_token_prob

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

        # sampling temperature
        self.temperature = temperature

        # loss weights
        self.disc_weight = disc_weight
        self.gen_weight = gen_weight

    def forward(self, input, **kwargs):
        b, t = input.shape

        replace_prob = prob_mask_like(input, self.replace_prob)

        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # set inverse of mask to padding tokens for labels
        gen_labels = input.masked_fill(~mask, self.pad_token_id)

        # clone the mask, for potential modification if random tokens are involved
        # not to be mistakened for the mask above, which is for all tokens, whether not replaced nor replaced with random tokens
        masking_mask = mask.clone()

        # if random token probability > 0 for mlm
        if self.random_token_prob > 0:
            assert self.num_tokens is not None, 'Number of tokens (num_tokens) must be passed to Electra for randomizing tokens during masked language modeling'

            random_token_prob = prob_mask_like(input, self.random_token_prob)
            random_tokens = torch.randint(0, self.num_tokens, input.shape, device=input.device)
            random_no_mask = mask_with_tokens(random_tokens, self.mask_ignore_token_ids)
            random_token_prob &= ~random_no_mask
            masked_input = torch.where(random_token_prob, random_tokens, masked_input)

            # remove random token prob mask from masking mask
            masking_mask = masking_mask & ~random_token_prob

        # [mask] input
        masked_input = masked_input.masked_fill(masking_mask * replace_prob, self.mask_token_id)
        # get generator output and get mlm loss
        logits,_ = self.generator(masked_input,return_all_hiddens=False)
        # logits=self.generator.to_logits(embs)

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2),
            gen_labels,
            ignore_index = self.pad_token_id
        )

        # use mask from before to select logits that need sampling
        sample_logits = logits[mask_indices]

        # sample
        sampled = gumbel_sample(sample_logits, temperature = self.temperature)

        # scatter the sampled values back to the input
        disc_input = input.clone()
        disc_input[mask_indices] = sampled.detach()

        # generate discriminator labels, with replaced as True and original as False
        disc_labels = (input != disc_input).float().detach()

        # get discriminator predictions of replaced / original
        non_padded_indices = torch.nonzero(input != self.pad_token_id, as_tuple=True)

        # get discriminator output and binary cross entropy loss
        # disc_logits = self.discriminator(disc_input, **kwargs)
        disc_embs,_ = self.discriminator(masked_input,features_only=True,return_all_hiddens=False)
        disc_logits=self.discriminator.to_binary(disc_embs)
        disc_logits = disc_logits.reshape_as(disc_labels)

        disc_loss = F.binary_cross_entropy_with_logits(
            disc_logits[non_padded_indices],
            disc_labels[non_padded_indices]
        )

        # gather metrics
        with torch.no_grad():
            gen_predictions = torch.argmax(logits, dim=-1)
            disc_predictions = torch.round((torch.sign(disc_logits) + 1.0) * 0.5)
            gen_acc = (gen_labels[mask] == gen_predictions[mask]).float().mean()
            disc_acc = 0.5 * (disc_labels[mask] == disc_predictions[mask]).float().mean() + 0.5 * (disc_labels[~mask] == disc_predictions[~mask]).float().mean()

        # return weighted sum of losses
        return Results(self.gen_weight * mlm_loss + self.disc_weight * disc_loss, mlm_loss, disc_loss, gen_acc, disc_acc, disc_labels, disc_predictions)


In [7]:
MASK_ID=4
PAD_ID=0
model = Electra(
    generator,
    discriminator,
    discr_dim = 768,           # the embedding dimension of the discriminator
    # discr_layer = 'reformer',   # the layer name in the discriminator, whose output would be used for predicting token is still the same or replaced
    discr_layer = 'attn_layers',   # the layer name in the discriminator, whose output would be used for predicting token is still the same or replaced
    mask_token_id = MASK_ID,          # the token id reserved for masking
    pad_token_id = PAD_ID,           # the token id for padding
    mask_prob = 0.15,           # masking probability for masked language modeling
    mask_ignore_token_ids = []  # ids of tokens to ignore for mask modeling ex. (cls, sep)
)

In [8]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)


In [10]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        # with accelerator.accumulate(model):
        results=model(batch['input_ids']) 

        count_amostra+=int(len(batch['input_ids']))
        loss = results.loss
        # print(loss)
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []


iteration:  0 , total_loss:  1.474444834391276
count_amostra: 200
iteration:  30 , total_loss:  13.770268281300863
count_amostra: 6200
